# Exploratory Data Analysis (EDA)

In order to get an idea of how the preprocess the airbnb data, we perform first an EDA by usind pandas profiling. 

For the EDA we load the latest version of the data `sample.csv` from Weights & Biases. Our project is called "nyc_airbnb". Note that we use `save_code=True` in the call of wandb.init, i.e. the notebook is uploaded and versioned by Weights & Biases.

In [1]:
import wandb
import pandas as pd

run = wandb.init(project="nyc_airbnb", group="eda", save_code=True)
local_path = wandb.use_artifact("sample.csv:latest").file()
df = pd.read_csv(local_path)

wandb: Currently logged in as: mariaf (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Next, we use `pandas_profiling` in order to get a first impression of the data.

In [2]:
import pandas_profiling

profile = pandas_profiling.ProfileReport(df)
profile.to_widgets()

Summarize dataset:   0%|          | 0/29 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

Here we notice, that there are missing values in a few columns and the column `last_review` is a date but it is in string format. Furthermore, the price column has a few outliers. There are some zeros and some very high prices. In order to fix that (and after talking to stakeholders) we decide to consider from a minimum of 10 Dollar to a maximum of 350 Dollar per night.

In [3]:
# Drop outliers
min_price = 10
max_price = 350
idx = df['price'].between(min_price, max_price)
df = df[idx].copy()
# Convert last_review to datetime
df['last_review'] = pd.to_datetime(df['last_review'])

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19001 entries, 0 to 19999
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              19001 non-null  int64         
 1   name                            18994 non-null  object        
 2   host_id                         19001 non-null  int64         
 3   host_name                       18993 non-null  object        
 4   neighbourhood_group             19001 non-null  object        
 5   neighbourhood                   19001 non-null  object        
 6   latitude                        19001 non-null  float64       
 7   longitude                       19001 non-null  float64       
 8   room_type                       19001 non-null  object        
 9   price                           19001 non-null  int64         
 10  minimum_nights                  19001 non-null  int64         
 11  nu

We did not impute missing values since it will be done in the inference pipeline, so we will be able to handle missing values also in production.

In [5]:
run.finish()